In [178]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#from sklearn.model_selection import train_test_split
import tensorflow as tf
import librosa
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
from scipy.io import wavfile
from tempfile import mktemp
import itertools

from datasets import load_dataset

In [179]:
#TODO
#1. Do the data normalizing thing. you know you gotta (Done!)
#2. I think you need a better loss function- something that accounts for getting closer
#3. Test different timesteps
#4. Test different epochs
#5. Test differet batch sizes

#Original: Timestep = 2 Epoch = 2(?) Batch = 2(?)

In [180]:
TIME_STEPS = 7
EPOCHS = 5
BATCH_SIZE = 5
OPTIMIZER = 'adam'
LOSS_FUNCTION = 'sparse_categorical_crossentropy'
LETTER_CONVERSION = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q','r','s','t','u','v','w','x','y','z',' ','.', '' ]
image_name = 0

In [181]:
ds = load_dataset("mozilla-foundation/common_voice_11_0", "en", split='train[:100]')

Found cached dataset common_voice_11_0 (/home/hetricke/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631)


In [182]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [183]:
def convertToNum(s):
    try: 
        # print("original letter: " + s)
        # print("conversion number: "+str(LETTER_CONVERSION.index(s)))
        # print("conversion letter: "+ convertToLetter(s))
        # print()
        return LETTER_CONVERSION.index(s)
    except:
        return 29

In [184]:
def convertToLetter(val):
    try:
        return LETTER_CONVERSION[val]
    except:
        return "'"

In [185]:
def transcript_prep(transcription):

    if transcription.startswith('"') and transcription.endswith('"'):
        # we can remove trailing quotation marks as they do not affect the transcription
        transcription = transcription[1:-1]

    transcription = transcription.lower()
    
    if len(transcription) > 0:
        if transcription[-1] not in [".", "?", "!"]:
            # append a full-stop to sentences that do not end in punctuation
            transcription = transcription + "."

    transcript_pieces = [convertToNum(c) for c in transcription]


    return transcript_pieces


In [186]:
def audio_file_prep(audio_path, sentence_length):

    global image_name

    #reads in mp3
    mp3_audio = AudioSegment.from_file(audio_path, format="mp3")  # read mp3

    if(round(mp3_audio.duration_seconds) == 0):
        return -1

    #removes silent audio from the beginning and end
    start_trim = detect_leading_silence(mp3_audio)
    end_trim = detect_leading_silence(mp3_audio.reverse())
    duration = len(mp3_audio)    
    trimmed_sound = mp3_audio[start_trim:duration-end_trim]

    #converts the mp3 into a wav file
    wname = mktemp('.wav')  # use temporary file
    trimmed_sound.export(wname, format="wav")  # convert to wav
    FS, audio_data = wavfile.read(wname)  # read wav file


    #creates a file name for the spectrogram
    file_name = "images/"+ str(image_name) + ".png"
    image_name = image_name + 1

    #creates and saves the spectrogram
    plt.figure()
    plt.specgram(audio_data, Fs=FS, NFFT=128, noverlap=0)  # plot
    plt.axis('off')
    plt.savefig(file_name, bbox_inches='tight')

    #clears the figure for the next audio transcript- otherwise it just overwrites the image, which causes Problems
    plt.close()


    #loads the spectrogram and turns it into an array
    img = keras.preprocessing.image.load_img(file_name)
    img_array = keras.preprocessing.image.img_to_array(img)


    #slices the image array into appriopriate sizes for each chunk
    features = int(img_array.shape[0]/sentence_length)
    
    divided_img_array = []
    x_data = []
    index = 0

    for i in range(sentence_length):
        feature_array = img_array[index:index+features]
        new_dim = feature_array.shape[0]*feature_array.shape[1]
        feature_array = feature_array.reshape(new_dim, -1)
        feature_array = feature_array.flatten()
        
        padding = 33000-feature_array.shape[0]
        feature_array = np.pad(feature_array, [(padding), (0)], mode='constant')

        feature_array = feature_array.tolist()
        feature_array[:] = [x / 255 for x in feature_array]

        divided_img_array.append(feature_array)

        index = index + features


    half_timestep = int(TIME_STEPS/2)

    for i in range(sentence_length):

        start = i-half_timestep
        end = i + half_timestep

        start = start if (start >= 0) else 0
        end = end if (end < len(divided_img_array)) else len(divided_img_array)-1

        if(end-start < TIME_STEPS):

            info = divided_img_array[start:end].copy()
            null_list = [0]*(len(feature_array))


            while len(info) < TIME_STEPS and start == 0:
                info.insert(0, null_list.copy())

            while len(info)<TIME_STEPS:
                info.append(null_list.copy())

    
            x_data.append(info.copy())

        else:
            x_data.append(divided_img_array[start:end].copy())

        
    


    #reshape the spectogram into a 3d array that goes (num samples x timesteps x features)
    #1 sample is how many times this gets run through
    #timesteps is how far backward to grab (so that's a value that can be played with!)
    #num samples is I think how long it is (so grab the mp3 length)
    #slice the image accordingly
    #convert the image into an array (reference the obj detection) to use for the features
    #profit

    #so the min time is three, which, at 515 timesteps, gets 171.6666 columns of data per second
    

    return x_data


In [187]:
image_name = 0
x_processed_data = []
y_processed_data = []

for i in range(25):

    x_processed_data.append(audio_file_prep(ds[i]['path'], len(ds[i]["sentence"])))
    y_processed_data.append(transcript_prep(ds[i]["sentence"]))

    if(x_processed_data[i] == -1 or len(y_processed_data[i])==0):
       x_processed_data.pop(i)
       y_processed_data.pop(i)


In [188]:
x_train = np.array(x_processed_data[:20])
y_train = np.array(y_processed_data[:20])

x_test = np.array(x_processed_data[20:25])
y_test = np.array(y_processed_data[20:25])

/tmp/ipykernel_102850/3883480599.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(x_processed_data[:20])
/tmp/ipykernel_102850/3883480599.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array(y_processed_data[:20])
/tmp/ipykernel_102850/3883480599.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarra

In [189]:
#build model
model = keras.Sequential()
model.add(keras.layers.Dense(units = 128, activation = 'relu', input_shape = (TIME_STEPS, 33000)))
model.add(keras.layers.Dense(units = 128, activation='relu'))
model.add(keras.layers.Dense(units = 128, activation ='relu'))
model.add(keras.layers.SimpleRNN(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(len(LETTER_CONVERSION)+1, activation = 'softmax'))
model.compile(optimizer=OPTIMIZER, metrics=['accuracy'], loss=LOSS_FUNCTION)

In [190]:
for recording in range(20):
    print(recording)
    model.fit(x_train[recording], y_train[recording], epochs=EPOCHS, batch_size=BATCH_SIZE)


0
Epoch 1/5
12/12 [==============================] - 2s 53ms/step - loss: 3.7920 - accuracy: 0.0357
Epoch 2/5
12/12 [==============================] - 1s 66ms/step - loss: 3.2730 - accuracy: 0.1607
Epoch 3/5
12/12 [==============================] - 1s 55ms/step - loss: 3.3279 - accuracy: 0.1250
Epoch 4/5
12/12 [==============================] - 1s 54ms/step - loss: 3.1863 - accuracy: 0.1071
Epoch 5/5
12/12 [==============================] - 1s 62ms/step - loss: 2.9757 - accuracy: 0.1607
1
Epoch 1/5
19/19 [==============================] - 1s 52ms/step - loss: 3.1353 - accuracy: 0.0978
Epoch 2/5
19/19 [==============================] - 1s 53ms/step - loss: 3.1134 - accuracy: 0.0761
Epoch 3/5
19/19 [==============================] - 1s 50ms/step - loss: 2.9647 - accuracy: 0.1196
Epoch 4/5
19/19 [==============================] - 1s 61ms/step - loss: 2.8818 - accuracy: 0.1522
Epoch 5/5
19/19 [==============================] - 1s 61ms/step - loss: 2.9023 - accuracy: 0.0978
2
Epoch 1/5
16/1

In [191]:
total_score = 0
total_accuracy = 0
for i in range(len(x_test)):
    score, accuracy = model.evaluate(x_test[i], y_test[i], batch_size=BATCH_SIZE)
    total_score += score
    total_accuracy += accuracy

total_score /= len(x_test)
total_accuracy /= len(x_test)

print(total_accuracy)

 7/12 [================>.............] - ETA: 0s - loss: 2.9194 - accuracy: 0.1714  

2023-12-02 16:51:20.751486: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_12/simple_rnn_12/while/body/_1/sequential_12/simple_rnn_12/while/simple_rnn_cell_12/Relu}}
	.  Registered:  <no registered kernels>

2023-12-02 16:51:20.760076: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:134] tfg_optimizer{any(tfg-consolidate-attrs,tfg-toposort,tfg-shape-inference{graph-version=0},tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential_12/simple_rnn_12/while/body/_1/sequential_12/simple_rnn_12/while/simple_rnn_cell_12/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2023-12-02 16:51:20.770148: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:134] tfg_optimizer{any(tfg-consolidate-attrs,tfg-functional-to-region,tfg.func(tfg-cf-sink),tfg-region-to-functional{force-control-capture=true},tfg-lift-legacy-call,symbol-privatize{},symbol-dce,tfg-

15/15 [==============================] - 0s 19ms/step - loss: 2.8910 - accuracy: 0.1268
0.11972274333238601
